## TensorRT基本介绍

TensorRT是一种高性能深度学习推理优化器和运行时加速库，可以为深度学习应用提供低延迟、高吞吐率的部署推理。

TensorRT现已能支持TensorFlow、Caffe、Mxnet、Pytorch等几乎所有的深度学习框架，将TensorRT和NVIDIA的GPU结合起来，能在几乎所有的框架中进行快速和高效的部署推理。


为了提高部署推理的速度，出现了很多模型优化的方法，如：模型压缩、剪枝、量化、知识蒸馏等，这些一般都是在训练阶段实现优化。

而TensorRT 则是对训练好的模型进行优化，通过优化网络计算图提高模型效率

<img src="../data/img/tensorRT_opt.png" style="zoom:50%"/>

<img src="../data/img/tensorRT_work.png" />


## TensorRT 优化
<img src="../data/img/tensorRT_opt_process.png" style="zoom:50%"/>

### 层间融合或张量融合（Layer & Tensor Fusion）

制约计算速度的CUDA核心计算张量，往往大量的时间也是浪费在CUDA核心的启动和对每一层输入/输出张量的读写操作上面，这造成了内存带宽的瓶颈和GPU资源的浪费。

TensorRT通过对层间的横向或纵向合并，使得层的数量大大减少，这样就可以一定程度的减少kernel launches和内存读写。

<img src="../data/img/tensorRT_opt_1.png" />

横向合并可以把卷积、偏置和激活层合并成一个CBR结构（convolution, bias, and ReLU layers are fused to form a single layer），只占用一个CUDA核心。

纵向合并可以把结构相同，但是权值不同的层合并成一个更宽的层，也只占用一个CUDA核心。

另外，对于多分支合并的情况，TensorRT完全可以实现直接接到需要的地方，不用专门做concat的操作，所以这一层也可以取消掉。

合并之后的计算图（右侧）的层次更少了，占用的CUDA核心数也少了，因此整个模型结构会更小，更快，更高效。


INT8只有256个不同的数值，使用INT8来表示 FP32精度的数值，肯定会丢失信息，造成性能下降。不过TensorRT会提供完全自动化的校准（Calibration ）过程，会以最好的匹配性能将FP32精度的数据降低为INT8精度，最小化性能损失。关于校准过程，后面会专门做一个探究。

- Kernel Auto-Tuning内核自动调整

网络模型在推理计算时，是调用GPU的CUDA核进行计算的。TensorRT可以针对不同的算法，不同的网络模型，不同的GPU平台，进行 一些Kernel层面的优化，以保证当前模型在特定平台上以最优性能计算。

- Dynamic Tensor Memory动态张量显存

在每个tensor使用期间，TensorRT会为其指定显存，避免显存重复申请，减少内存占用和提高内存的重复使用效率（reuse）。

- Multi-Stream Execution多流执行

使用CUDA中的stream技术，对于同一输入的多个分支可以进行并行运算，还可以根据不同batchsize优化。

## TensorRT 工作方式

<img src="../data/img/tensorRT_work_process.png" style="zoom:50%"/>
基本流程：

1、训练神经网络，

2、优化得到推理引擎plan，序列化到磁盘

3、使用时反序列化，使用优化后的推理引擎进行加速

<img src="../data/img/tensorRT_work_process_1.png" style="zoom:50%"/>

1、创建builder：构造器。包含tensorrt所有组件，及推理的整个流程，网络、解析器、输入输出的buffer地址，推理优化的引擎，输入输出结果的维度等

2、创建network。保存训练好的神经网络。

3、创建parser：用来解析网络。caffe、tensorflow、onnx……也可自定义
4、绑定：gpu的输入、输出地址、自定义组件

 

输入：训练好的神经网络模型及其结构。支持所有框架。

输出：一个可执行的推理引擎。

5、序列化反序列化

6、传输计算数据：CPU到GPU

7、执行计算

8、传输计算结果：GPU到CPU